# Interactive Wind Load Analysis (ASCE 7-16)

This notebook interactively determines if your building is eligible for the **Simplified Procedure for Low-Rise Buildings (ASCE 7-16, Chapter 28, Part 1)**.

You will be prompted to answer a series of questions about the building's geometry, site, and classification. You must meet **all criteria** to proceed to the simplified wind pressure calculation.

In [13]:
# Import necessary libraries
import ipywidgets as widgets
from IPython.display import display, clear_output

# --- Engineering Data and Functions (from previous script) ---

def get_kh_coefficient(height, exposure):
    """Looks up the Velocity Pressure Exposure Coefficient (Kh) from a table."""
    kh_table = {
        'B': {15: 0.57, 20: 0.62, 25: 0.66, 30: 0.70, 40: 0.76, 50: 0.81, 60: 0.85},
        'C': {15: 0.85, 20: 0.90, 25: 0.94, 30: 0.98, 40: 1.04, 50: 1.09, 60: 1.13},
        'D': {15: 1.03, 20: 1.08, 25: 1.12, 30: 1.16, 40: 1.22, 50: 1.27, 60: 1.31}
    }
    valid_heights = sorted(kh_table[exposure].keys())
    lookup_height = next((h for h in valid_heights if h >= height), max(valid_heights))
    return kh_table[exposure][lookup_height]

# Initialize a dictionary to store all validated user inputs
building_params = {}

print("✅ Libraries and helper functions loaded. Proceed to the next cell.")

✅ Libraries and helper functions loaded. Proceed to the next cell.


In [14]:
# --- Create Interactive Widgets for Eligibility Check ---
style = {'description_width': 'initial'}
layout = widgets.Layout(width='600px')

# Create widgets for each criterion
w_height = widgets.IntText(description="Mean Roof Height (h) in ft:", style=style, layout=layout)
w_risk_cat = widgets.Dropdown(options=['I or II', 'III', 'IV'], description="Risk Category:", style=style, layout=layout)
w_building_type = widgets.Dropdown(options=['Simple Diaphragm Building', 'Other'], description="Building Type:", style=style, layout=layout)
w_topography = widgets.Dropdown(options=['Flat Terrain (Kzt = 1.0)', 'Hills/Ridges (Kzt > 1.0)'], description="Topography:", style=style, layout=layout)
w_roof_shape = widgets.Dropdown(options=['Flat, Gable, or Hip', 'Other/Complex'], description="Roof Shape:", style=style, layout=layout)
w_roof_slope = widgets.FloatText(description="Roof Slope (θ) in degrees:", style=style, layout=layout)

# Create a button to trigger the check
check_button_1 = widgets.Button(description="Check Eligibility", button_style='info')

# Create an output area to display messages
output_1 = widgets.Output()

def check_eligibility_part1(b):
    """Validates user inputs against ASCE 7-16, Ch. 28 criteria."""
    output_1.clear_output()
    
    errors = []
    # --- Validation Logic ---
    if not w_height.value > 0:
        errors.append("Height must be greater than 0.")
    elif w_height.value > 60:
        errors.append(f"Height ({w_height.value} ft) exceeds the 60 ft limit.")
    
    if w_risk_cat.value != 'I or II':
        errors.append("Risk Category must be I or II.")
        
    if w_building_type.value != 'Simple Diaphragm Building':
        errors.append("Building must be a Simple Diaphragm Building.")
        
    if w_topography.value != 'Flat Terrain (Kzt = 1.0)':
        errors.append("Site must have flat terrain (Kzt = 1.0).")
        
    if w_roof_shape.value != 'Flat, Gable, or Hip':
        errors.append("Roof must be Flat, Gable, or Hip.")
        
    if not w_roof_slope.value >= 0:
         errors.append("Roof slope cannot be negative.")
    elif w_roof_slope.value > 45:
        errors.append(f"Roof slope ({w_roof_slope.value}°) exceeds the 45° limit.")
    
    with output_1:
        if not errors:
            print("✅ SUCCESS: Building is ELIGIBLE for the Simplified Procedure.")
            print("You may proceed to the next step to enter site data and run the calculation.")
            # Store the validated height for later use
            building_params['mean_roof_height_h'] = w_height.value
        else:
            print("❌ INELIGIBLE: Your building does not meet the criteria for the Simplified Procedure.")
            print("Reasons:")
            for error in errors:
                print(f"  - {error}")
            print("\nThis method CANNOT be used. A more detailed analysis (e.g., ASCE 7-16 Chapter 27) is required.")

# Link button click to the function
check_button_1.on_click(check_eligibility_part1)

# Display the widgets
print("Step 1: Determine if your building is eligible for the simplified procedure.")
display(widgets.VBox([
    w_height, w_risk_cat, w_building_type, w_topography, w_roof_shape, w_roof_slope,
    check_button_1, output_1
]))

Step 1: Determine if your building is eligible for the simplified procedure.


In [15]:
# --- Create widgets for the final calculation step ---
style = {'description_width': 'initial'}
layout = widgets.Layout(width='600px')

w_wind_speed = widgets.IntText(description="Basic Wind Speed (V) in mph:", style=style, layout=layout)
w_exposure = widgets.Dropdown(options=['B', 'C', 'D'], description="Exposure Category:", style=style, layout=layout)
w_enclosure = widgets.Dropdown(options=['Enclosed', 'Partially Enclosed'], description="Enclosure Classification:", style=style, layout=layout)
w_parapets = widgets.Dropdown(options=['No Parapets', 'Has Parapets'], description="Parapets:", style=style, layout=layout)

# Create a button to run the calculation
calc_button = widgets.Button(description="Calculate Wind Pressures", button_style='success')
output_2 = widgets.Output()

def run_calculation(b):
    """Performs the final wind pressure calculation after checking script limitations."""
    output_2.clear_output()
    
    # Check if the building was ever validated in the previous step
    if 'mean_roof_height_h' not in building_params:
        with output_2:
            print("❗️ Please go back to Step 1 and successfully validate your building's eligibility first.")
        return

    # Check this script's specific limitations
    errors = []
    if w_parapets.value == 'Has Parapets':
        errors.append("This script is not configured to handle parapets.")
    if w_wind_speed.value <= 0:
        errors.append("Wind speed must be a positive value.")

    if errors:
        with output_2:
            print("❌ ERROR: Cannot run calculation due to script limitations.")
            for error in errors:
                print(f"  - {error}")
        return

    # --- If all checks pass, proceed with calculation ---
    with output_2:
        print("🚀 Running calculations...")
        # Collect all parameters
        h = building_params['mean_roof_height_h']
        V = w_wind_speed.value
        exposure = w_exposure.value
        enclosure = w_enclosure.value
        
        # Perform Calculations
        Kh = get_kh_coefficient(h, exposure)
        qh = 0.00256 * Kh * 1.0 * 1.0 * (V**2) # Kzt and Ke are 1.0 per eligibility check
        
        gcpf_values = {'windward_wall': 0.4, 'leeward_wall': -0.29, 'side_wall': -0.45}
        
        if enclosure == 'Enclosed':
            gcpi_pos, gcpi_neg = 0.18, -0.18
        else: # Partially Enclosed
            gcpi_pos, gcpi_neg = 0.55, -0.55
            
        # Calculate pressure envelope for each surface
        results = {}
        for wall, gcpf in gcpf_values.items():
            p_pos = qh * (gcpf - gcpi_pos)
            p_neg = qh * (gcpf - gcpi_neg)
            results[wall] = (min(p_pos, p_neg), max(p_pos, p_neg))

        # --- Display Final Results ---
        print("\n==========================================================")
        print("           DESIGN WIND PRESSURE RESULTS (MWFRS)           ")
        print("==========================================================")
        print(f"INPUTS:\n  Wind Speed: {V} mph | Height: {h} ft | Exposure: '{exposure}' | Enclosure: '{enclosure}'\n")
        print(f"CALCULATED:\n  Kh: {Kh:.2f} | qh: {qh:.2f} psf\n")
        print("----------------------------------------------------------")
        print("DESIGN PRESSURE ENVELOPES (psf):")
        print("(+) is pressure towards surface, (-) is suction away.\n")
        
        for wall, (p_min, p_max) in results.items():
            print(f"  {wall.replace('_', ' ').title():<15} [{p_min:6.2f}, {p_max:6.2f}] psf")
        print("==========================================================")


# Link button to function
calc_button.on_click(run_calculation)

print("\nStep 2: If eligible, provide site details and calculate pressures.")
display(widgets.VBox([w_wind_speed, w_exposure, w_enclosure, w_parapets, calc_button, output_2]))


Step 2: If eligible, provide site details and calculate pressures.


In [ ]:
# PRINTABLE SUMMARY - Your actual calculation results
# This creates a nicely formatted summary that will print well to PDF

from IPython.display import display, Markdown
from datetime import datetime

# Your actual calculation results from the terminal
V = 115  # Wind Speed (mph)
h = 25   # Mean Roof Height (ft)  
exposure = 'B'  # Exposure Category
Kh = 0.66  # Exposure Coefficient
qh = 22.34  # Velocity Pressure (psf)

# Your actual pressure results (psf)
windward_wall = (4.92, 12.96)
leeward_wall = (-10.50, -2.46)
side_walls = (-14.08, -6.03)

# Create the formatted summary
summary = f"""
# Wind Load Calculation Results
**ASCE 7-16 Chapter 28 - Simplified Procedure for Low-Rise Buildings**

---

## Input Parameters
- **Basic Wind Speed (V):** {V} mph
- **Mean Roof Height (h):** {h} ft  
- **Exposure Category:** {exposure}

## Calculated Values
- **Velocity Pressure Exposure Coefficient (Kh):** {Kh:.2f}
- **Velocity Pressure (qh):** {qh:.2f} psf

## Design Pressure Envelopes (psf)

| Surface | Minimum Pressure | Maximum Pressure | Design Range |
|---------|------------------|------------------|--------------|
| **Windward Wall** | {windward_wall[0]:.2f} psf | {windward_wall[1]:.2f} psf | [{windward_wall[0]:.2f}, {windward_wall[1]:.2f}] |
| **Leeward Wall** | {leeward_wall[0]:.2f} psf | {leeward_wall[1]:.2f} psf | [{leeward_wall[0]:.2f}, {leeward_wall[1]:.2f}] |
| **Side Walls** | {side_walls[0]:.2f} psf | {side_walls[1]:.2f} psf | [{side_walls[0]:.2f}, {side_walls[1]:.2f}] |

---

### Notes
- **Positive (+)** values indicate pressure **towards** the surface (pushing)
- **Negative (-)** values indicate pressure **away** from the surface (suction)  
- These values represent the design pressure envelopes for structural analysis
- Calculated per ASCE 7-16 Simplified Procedure for Low-Rise Buildings

**Report Generated:** {datetime.now().strftime('%B %d, %Y at %I:%M %p')}
"""

# Display the formatted summary
display(Markdown(summary))